In [2]:
# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [3]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyIEEM.models.utils import initialize_model

In [4]:
country = 'SWE'
spatial = True
age_classes = pd.IntervalIndex.from_tuples([(0,5),(5,10),(10,15),(15,20),(20,25),(25,30),(30,35),(35,40),(40,45),(45,50),(50,55),(55,60),(60,65),(65,70),(70,75),(75,80),(80,120)], closed='left')

In [5]:
def compute_beta(R0, delta, gamma, s, N, demography):
    beta_i = R0/(s*(delta+gamma)*np.expand_dims(np.sum(N, axis=0),axis=1))
    return sum(np.squeeze(beta_i)*demography/sum(demography))

In [9]:
N = np.squeeze(model.parameters['N']['other'] + model.parameters['G']*model.parameters['N']['work'])
demography = pd.read_csv(f'../data/interim/epi/demographic/age_structure_{country}_2019.csv', index_col=[0, 1]).groupby(by='age').sum().squeeze()
demography = demography.groupby(pd.cut(demography.index.values, age_classes)).sum()
compute_beta(3.3, model.parameters['delta'], model.parameters['gamma'], model.parameters['s'], N, demography.values)

ValueError: operands could not be broadcast together with shapes (11,11) (17,17,11) 

In [99]:
simulation_start = '2019-12-15'
model = initialize_model(country, age_classes, spatial, simulation_start, contact_type='absolute_contacts')

In [ ]:
fig,ax = plt.subplots()

model.parameters['beta'] = 0.028
model.parameters['amplitude'] = 0.20

# economic lockdown, schools closed, preventive measures, social restrictions
model.parameters['economic_closures'].loc[slice(None)] = 1
out = model.sim([simulation_start, '2020-12-01'])
ax.plot(out.date, out.Ih.sum(dim=['age_class', 'spatial_unit']), color='green', label='lockdown')
#ax.plot(out.date, out.R.sum(dim=['age_class', 'spatial_unit']), color='green', label='lockdown')

# economic lockdown, schools open, preventive measures, social restrictions
#model.parameters['economic_closures'].loc['P85'] = 0
#out = model.sim([simulation_start, '2020-12-01'])
#ax.plot(out.date, out.Hin.sum(dim=['age_class', 'spatial_unit']), 'orange', label='lockdown with\nschools open')

# do-nothing
#model.parameters['social_restrictions'] = 0
#model.parameters['economic_closures'].loc[slice(None)] = 0
#out = model.sim([simulation_start, '2020-12-01'])
#ax.plot(out.date, out.Hin.sum(dim=['age_class', 'spatial_unit']), color='red', label='do nothing')

ax.legend()

ax.axvline(x=datetime(2020, 3, 15), color='black', linestyle='--', linewidth=1)
ax.set_xlabel('date')
ax.set_ylabel('Infected (-)')
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
plt.show()
plt.close()

2019-12-15 00:27:47.449834 0.0 0.0
2019-12-15 00:31:08.772489 0.0 0.0
2019-12-15 00:41:31.696652 2.375448215459191e-08 1.3597748247739665e-09
2019-12-15 04:09:10.179912 3.734875512069553e-07 2.170527957655901e-08
2019-12-15 05:52:59.421542 5.747155941899253e-06 3.1996071727850456e-07
2019-12-15 07:36:48.663172 3.0121358831778512e-05 1.8027750314650898e-06
2019-12-15 13:15:10.464117 8.66384221065583e-05 5.888609852877265e-06
2019-12-15 16:04:21.364589 0.0002308701099625664 1.630932111593082e-05
2019-12-15 18:53:32.265062 0.00035089978139523584 3.1240117394854294e-05
2019-12-16 03:20:46.094828 0.0007444865807249581 6.245083462366133e-05
2019-12-16 07:34:23.009711 0.00136301837225987 0.00011763695512121893
2019-12-16 11:47:59.924594 0.0017435126304196171 0.00018462173773336403
2019-12-17 00:54:44.288872 0.003225964360512589 0.0003095571371011478
2019-12-17 07:28:06.471011 0.005005694472555307 0.0004983622249710676
2019-12-17 14:01:28.653151 0.005960009902057379 0.000713452823414006
2019-1

2020-02-07 13:37:54.107764 13.988312378105416 4.69913303372024
2020-02-08 00:53:57.117991 15.170356015948618 5.14801240020534
2020-02-08 12:10:00.128219 16.167961834183664 5.615789789021798
2020-02-09 10:57:31.460817 18.396935182728942 6.171176196747464
2020-02-09 22:21:17.127116 19.974912221152366 6.764995408706073
2020-02-10 09:45:02.793415 21.296746885234942 7.384147199124755
2020-02-11 08:03:41.659723 24.172417846659712 8.11526199257421
2020-02-11 19:13:01.092877 26.18638144938613 8.893761038535674
2020-02-12 06:22:20.526031 27.888778729447743 9.703621761351243
2020-02-13 04:58:48.228976 31.694682501530657 10.662735867467859
2020-02-13 16:17:02.080449 34.37728756790998 11.68624245518657
2020-02-14 03:35:15.931921 36.627240310365615 12.751940347264144
2020-02-15 01:52:59.457393 41.55323526306213 14.009593040706356
2020-02-15 13:01:51.220128 44.99486991604041 15.347848180473154
2020-02-16 00:10:42.982864 47.90598382849197 16.73934195100523
2020-02-16 22:55:49.344065 54.45627480943452

2020-04-04 18:17:48.996223 10068.088085430414 6305.6990488670435
2020-04-05 04:16:54.059499 10186.117723552448 6457.497667117912
2020-04-06 03:30:59.057440 10445.312681866173 6617.941126816503
2020-04-06 15:08:01.556411 10544.04641147405 6774.919381259855
2020-04-07 02:45:04.055382 10663.775033310387 6929.482779880363
2020-04-05 20:39:40.317982 10362.02243071974 7062.7594939054725
2020-04-06 02:07:15.737476 10420.700833422576 7190.24465721487
2020-04-05 17:37:05.461015 10330.373726330039 7306.573042794333
2020-04-05 22:03:49.261521 10378.482541493886 7417.719222502279
2020-04-06 06:57:16.862532 10471.661679548459 7527.112498168046
2020-04-06 11:24:00.663037 10513.767034175997 7631.723256696273
2020-04-06 15:50:44.463543 10558.86917850831 7732.054166328249
2020-04-07 00:52:54.598787 10647.501929158789 7831.218095237066
2020-04-07 05:23:59.666409 10687.373585937617 7926.3977014225275
2020-04-07 09:55:04.734031 10730.28492817253 8018.006284669917
2020-04-08 14:30:19.843484 10992.469297413

2020-06-11 18:43:26.511951 4485.791290579664 6478.5027320373865
2020-06-12 04:54:32.445701 4445.539616398682 6396.817172694164
2020-06-12 15:05:38.379451 4401.7147767228225 6317.471951283423
2020-06-13 11:51:45.514604 4316.009414701943 6236.23823486754
2020-06-13 22:14:49.082181 4278.33592211879 6157.569986218038
2020-06-14 08:37:52.649758 4237.209857167663 6081.153570824469
2020-06-15 05:25:59.340675 4158.213683286908 6003.056694172523
2020-06-15 15:50:02.686133 4123.880270178963 5927.499968804043
2020-06-16 02:14:06.031591 4085.9937693394304 5854.172484907616
2020-06-16 23:21:50.042070 4012.645867104088 5779.26149998271
2020-06-17 09:55:42.047310 3981.069417293808 5706.817957957831
2020-06-17 20:29:34.052549 3946.082272255615 5636.539463951768
2020-06-18 17:09:13.597110 3880.9251304987633 5565.055438537369
2020-06-19 03:29:03.369390 3853.1730383241816 5496.072282998316
2020-06-19 13:48:53.141670 3822.0989853488886 5429.285921105778
2020-06-20 10:36:06.226594 3762.920394359703 5361.43